#### Library Imports

In [2]:
# Supress Warnings
import warnings
import pickle
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Machine Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, ConvLSTM2D, BatchNormalization
import tensorflow as tf
import keras

# Planetary Computer Tools
import pystac_client
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('58d635c0d3194d65b98b857cf6966e7f')

# Others
from itertools import cycle
from tqdm import tqdm   
import joblib
tqdm.pandas()

#### Functions

In [3]:
# Process datasets to be uniform in the number of timsteps and pixels per sample 
def reshapeData3D(dataset, pixel, instances):
    dataset_reshaped1 = []
    for i in dataset:
        if i.shape[2] > pixel and i.shape[1] > pixel:
            dataset_reshaped1.append(i[:,:(pixel-i.shape[1]),:(pixel-i.shape[2])])
        elif i.shape[1] > pixel:
            dataset_reshaped1.append(i[:,:(pixel-i.shape[1]),:])
        elif i.shape[2] > pixel:
            dataset_reshaped1.append(i[:,:,:(pixel-i.shape[2])])
        else:
            dataset_reshaped1.append(i)

    dataset_reshaped = []
    for i in dataset_reshaped1:
        if i.shape[0] >instances:
            dataset_reshaped.append(i[:instances,:,:])
        else:
            dataset_reshaped.append((i[:,:,:]))
    return [np.transpose(i) for i in dataset_reshaped]

In [4]:
# Opens pickled data from scraped files
def read_file(month):
    with open(f'{month}.pkl', 'rb') as file:
        return pickle.load(file)

In [5]:
# Extracts data for each sample from time_of_interest, coordinates, and boxsize/resolution
def getData(time_of_interest, lat_long, box_size_deg, resolution):
    latlong=lat_long.replace('(','').replace(')','').replace(' ','').split(',')
    bbox = (float(latlong[1])-box_size_deg/2, float(latlong[0])-box_size_deg/2, float(latlong[1])+box_size_deg/2, float(latlong[0])+box_size_deg/2)
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace)
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = list(search.get_all_items())
    scale = resolution / 111320.0 
    data = stac_load(items,bands=["vv", 'vh'], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale)
    VV = np.array(data.vv)
    VH = np.array(data.vh)
    return VV, VH

#### Data Scraping (Deployed on Microsoft Planetary Computer)

In [27]:
# Extract coordinate samples from training and test sets
df = list(pd.read_csv('Crop_Location_Data.csv')['Latitude and Longitude'])
df2 = list(pd.read_csv('challenge_1_submission_template.csv')['id'])

#Set coordinates for data scraping
coordinates = df + df2
# Yields 24 timesteps of the same sample
time_of_interest = "2022-04-01/2022-08-31" 
#Yields a closeup, 90x90 pixel image per sample
box_size_deg = 0.008 
resolution = 10 

#Scraping data from planetary computer
VV, VH = [], []
for i in coordinates:
    vv, vh = getData(time_of_interest, i, box_size_deg, resolution)
    VV.append(vv)
    VH.append(vh)
    print(len(VH))

#Saving data from planetary computer into pickle files as lists
with open("APR-AUG_VV.pkl", 'wb') as save:
    pickle.dump(VV, save)
with open("APR-AUG_VH.pkl", 'wb') as save:
    pickle.dump(VH, save)

KeyboardInterrupt: 

#### Preprocessing

In [6]:
#Standardising the shape of VV and VH data samples
VH = reshapeData3D(read_file('APR-AUG_VH'), 90, 24)
VV = reshapeData3D(read_file('APR-AUG_VV'), 90, 24)

#Performing matrix calculations to derive the RGB values for each sample
RGB = []
for i in range(len(VV)):
    RGB.append(np.reshape(np.stack([VH[i], VV[i], np.divide(VH[i], VV[i])]), (1,24, 90, 90, 3)))

In [7]:
#Reshaping training data to represent the tuple: (number of samples, timesteps, dimensions, pixels X, pixels Y)
X = np.array(RGB).reshape(850, 24, 3, 90, 90)
X.shape

(850, 24, 3, 90, 90)

In [8]:
#Encoding Labels
oe = LabelEncoder()
y = oe.fit_transform(pd.read_csv('Crop_Location_Data.csv')['Class of Land'])

In [9]:
#Splitting Data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X[:600], y[:600], test_size=0.2, random_state=42)

#### Model Fitting and Training

In [84]:
#Define a tensorflow sequential network
model = Sequential()

#1st layer of Convolutional LSTM 
model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4]), padding='same', return_sequences=True))
model.add(BatchNormalization(axis=(2,3)))

#2nd layer of Convolutional LSTM 
model.add(ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=(2,3)))

#3rd layer of Convolutional LSTM 
model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=(2,3)))

#Additional Bath Normalisation 
model.add(BatchNormalization(axis=(2,3)))

#Flatten network results 
model.add(Flatten())

#Feed flattened data through a dense sigmoid neuron to produce a binary inference
model.add(Dense(units=1, activation='sigmoid'))

#Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [85]:
#Early stopping in the case of overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

#Fit the model to training and validation sets
history = model.fit(X_train, y_train, epochs=10, batch_size=5, validation_data=(X_val, y_val), callbacks=[early_stopping])

#Save the model as a pickle file
model.save('ConvLSTM_1.0')

Epoch 1/10


2023-03-11 17:58:45.656092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - ETA: 0s - loss: 12.9351 - accuracy: 0.6979

2023-03-11 18:02:52.934547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - 259s 3s/step - loss: 12.9351 - accuracy: 0.6979 - val_loss: 10.9191 - val_accuracy: 0.5167
Epoch 2/10
96/96 [==============================] - 243s 3s/step - loss: 6.6668 - accuracy: 0.8292 - val_loss: 1.3906 - val_accuracy: 0.9417
Epoch 3/10
96/96 [==============================] - 228s 2s/step - loss: 6.7751 - accuracy: 0.8792 - val_loss: 0.9881 - val_accuracy: 0.9917
Epoch 4/10
96/96 [==============================] - 228s 2s/step - loss: 2.1295 - accuracy: 0.9625 - val_loss: 3.3249 - val_accuracy: 0.9667
Epoch 5/10
96/96 [==============================] - 231s 2s/step - loss: 1.9519 - accuracy: 0.9688 - val_loss: 0.6651 - val_accuracy: 0.9583
Epoch 6/10
96/96 [==============================] - 234s 2s/step - loss: 4.4133 - accuracy: 0.9521 - val_loss: 2.4687 - val_accuracy: 0.9667
Epoch 7/10
96/96 [==============================] - 236s 2s/step - loss: 3.1074 - accuracy: 0.9563 - val_loss: 5.4202 - val_accuracy: 0.9250
Epoch 8/10
96/96 [====

#### Results Rendering

In [10]:
#Load the model to predict test samples
model =  keras.models.load_model('ConvLSTM_1.0')
y_pred = model.predict(X[600:])

#Convert the binary inference of the model into discrete labels; if 1 = Rice, if 0 = Not Rice)
result = []
for i in y_pred:
    if i>0.5:
        result.append('Rice')
    else:
        result.append('Non Rice')

#Load the test sample training dataimage.png
submission = pd.read_csv('challenge_1_submission_template.csv')
submission['target'] = result
submission.to_csv("1.0tests.csv")
submission

2023-03-12 01:14:38.401332: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-12 01:14:38.401962: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-03-12 01:14:44.994963: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-12 01:14:45.322361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 6s 468ms/step


,id,target
0,"(10.18019073690894, 105.32022315786804)",Rice
1,"(10.561107033461816, 105.12772097986661)",Rice
2,"(10.623790611954897, 105.13771401411867)",Rice
3,"(10.583364246115156, 105.23946127195805)",Non Rice
4,"(10.20744446668854, 105.26844107128906)",Rice
...,...,...
245,"(10.308283266873062, 105.50872812216863)",Non Rice
246,"(10.582910017285496, 105.23991550078767)",Non Rice
247,"(10.581547330796518, 105.23991550078767)",Non Rice
248,"(10.629241357910818, 105.15315779432643)",Rice
